In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Dashboard Configuration Setup
# MAGIC 
# MAGIC This notebook:
# MAGIC 1. Reads TML files from a volume folder
# MAGIC 2. Identifies all liveboards
# MAGIC 3. Creates/updates a configuration table with dashboard name, GUID, and process flag (default: N)

# COMMAND ----------

import json
import yaml
from pathlib import Path

# COMMAND ----------

# Configuration
CATALOG = "ds_training_1"
SCHEMA = "thoughtspot_inventory_ak"
VOLUME = "lvdash_files_ak"
FOLDER = "liveboard"

# Full path to TML files
TML_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/{FOLDER}/"

# Configuration table
CONFIG_TABLE = f"{CATALOG}.{SCHEMA}.liveboard_migration_config"

print(f"Reading from: {TML_PATH}")
print(f"Config table: {CONFIG_TABLE}")

# COMMAND ----------

def parse_tml_file(file_path):
    """Parse TML file (YAML or JSON)"""
    try:
        content = dbutils.fs.head(file_path, 10 * 1024 * 1024)
        try:
            return yaml.safe_load(content)
        except yaml.YAMLError:
            return json.loads(content)
    except Exception as e:
        print(f"  Error parsing file: {e}")
        return None

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 1: Read files from volume

# COMMAND ----------

try:
    all_files = dbutils.fs.ls(TML_PATH)
    tml_files = [f.path for f in all_files if f.path.endswith(('.tml', '.yaml', '.json'))]
    print(f"Found {len(tml_files)} TML files")
except Exception as e:
    print(f"ERROR: Cannot read from {TML_PATH}")
    print(f"Error: {e}")
    raise

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 2: Identify liveboards

# COMMAND ----------

liveboards = []

for file_path in tml_files:
    filename = Path(file_path).name
    print(f"Processing: {filename}")
    
    tml_data = parse_tml_file(file_path)
    
    if tml_data and 'liveboard' in tml_data:
        liveboard = tml_data['liveboard']
        name = liveboard.get('name', filename.replace('.tml', '').replace('.yaml', '').replace('.json', ''))
        guid = tml_data.get('guid', 'NO_GUID')
        
        liveboards.append({
            'name': name,
            'guid': guid,
            'process_flag': 'N'
        })
        
        print(f"  ✓ Found liveboard: {name} ({guid})")
    else:
        print(f"  ✗ Not a liveboard or parse error")

print(f"\nTotal liveboards found: {len(liveboards)}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 3: Create table and insert entries

# COMMAND ----------

# Create table if not exists
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")

create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {CONFIG_TABLE} (
    name STRING,
    guid STRING,
    process_flag STRING
) USING DELTA
"""

spark.sql(create_table_sql)
print(f"✓ Table {CONFIG_TABLE} ready")

# COMMAND ----------

# Get existing GUIDs
try:
    existing_df = spark.sql(f"SELECT guid FROM {CONFIG_TABLE}").toPandas()
    existing_guids = set(existing_df['guid'].tolist())
    print(f"Found {len(existing_guids)} existing entries")
except:
    existing_guids = set()
    print("No existing entries")

# COMMAND ----------

# Insert only new liveboards
import pandas as pd

new_liveboards = [lb for lb in liveboards if lb['guid'] not in existing_guids]

if new_liveboards:
    df = pd.DataFrame(new_liveboards)
    spark_df = spark.createDataFrame(df)
    spark_df.write.mode("append").saveAsTable(CONFIG_TABLE)
    print(f"✓ Added {len(new_liveboards)} new liveboards")
else:
    print("✓ No new liveboards to add")

# COMMAND ----------

# MAGIC %md
# MAGIC ## View Configuration Table

# COMMAND ----------

display(spark.sql(f"SELECT * FROM {CONFIG_TABLE} ORDER BY name"))

# COMMAND ----------

# MAGIC %md
# MAGIC ## Summary

# COMMAND ----------

summary = spark.sql(f"""
SELECT 
    process_flag,
    COUNT(*) as count
FROM {CONFIG_TABLE}
GROUP BY process_flag
""")

print("=== Summary ===")
display(summary)

print(f"""
Configuration table: {CONFIG_TABLE}

To enable a dashboard:
UPDATE {CONFIG_TABLE} 
SET process_flag = 'Y'
WHERE name = 'Your Dashboard Name';

To enable all:
UPDATE {CONFIG_TABLE} 
SET process_flag = 'Y';
""")

In [0]:
%sql
UPDATE ds_training_1.thoughtspot_inventory_ak.liveboard_migration_config 
SET process_flag = 'Y'
WHERE name = 'Different_visuals_dash';

In [0]:
%sql
use catalog 'ds_training_1';

In [0]:
%sql
select * from ds_training_1.thoughtspot_inventory_ak.liveboard_migration_config 